In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('humans.csv', header=0)
data

,ID,Age,Kid,Training,Minmax,Fruit,PlayerStarts,GameN,Length,Human,AI,Score
0,1,52,0,0,1,0,1,1,99,4,6,0.0
1,1,52,0,0,1,0,1,2,94,6,9,0.0
2,1,52,0,0,1,0,1,3,126,5,7,0.0
3,1,52,0,0,1,0,1,4,109,6,7,0.0
4,1,52,0,0,1,0,1,5,134,9,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
65,5,22,0,0,1,1,0,1,32,6,9,0.0
66,5,22,0,0,1,1,0,2,17,2,9,0.0
67,5,22,0,0,1,1,0,3,26,2,9,0.0
68,5,22,0,0,1,1,0,4,23,1,10,0.0
